In [2]:
# noexport

import os
os.system('export_notebook reconstruct_spans_persecond_xgboost.ipynb')

0

In [2]:
from tmilib import *

from reconstruct_focus_times_common import *
from sorted_collection import SortedCollection
from rescuetime_utils import *

import sklearn
import sklearn.svm
import sklearn.linear_model
import sklearn.ensemble
import sklearn.naive_bayes
import sklearn.tree
import xgboost

from math import log

import datetime
import time

import cPickle as pickle

In [ ]:
import traceback
import sys

In [3]:
training_users = get_training_users()
test_users = get_test_users()

In [4]:
#print domain_to_id('www.facebook.com')
#print id_to_domain(29708)

In [5]:


def extract_secondlevel_dataset_from_users(users):
  all_labels = []
  all_sinceprev = []
  all_tonext = []
  all_fromdomain = []
  all_todomain = []
  for user in users:
    data = extract_secondlevel_dataset_from_user(user)
    all_labels.extend(data['label'])
    all_sinceprev.extend(data['sinceprev'])
    all_tonext.extend(data['tonext'])
    all_fromdomain.extend(data['fromdomain'])
    all_todomain.extend(data['todomain'])
  return {
    'label': all_labels,
    'sinceprev': all_sinceprev,
    'tonext': all_tonext,
    'fromdomain': all_fromdomain,
    'todomain': all_todomain,
  }

@jsonmemoized
def extract_secondlevel_dataset_for_training():
  return extract_secondlevel_dataset_from_users(training_users)

@jsonmemoized
def extract_secondlevel_dataset_for_test():
  return extract_secondlevel_dataset_from_users(test_users)


In [6]:
def extract_tensecondlevel_dataset_from_users(users):
  all_labels = []
  all_sinceprev = []
  all_tonext = []
  all_fromdomain = []
  all_todomain = []
  for user in users:
    data = extract_secondlevel_dataset_from_user(user, True)
    all_labels.extend(data['label'])
    all_sinceprev.extend(data['sinceprev'])
    all_tonext.extend(data['tonext'])
    all_fromdomain.extend(data['fromdomain'])
    all_todomain.extend(data['todomain'])
  return {
    'label': all_labels,
    'sinceprev': all_sinceprev,
    'tonext': all_tonext,
    'fromdomain': all_fromdomain,
    'todomain': all_todomain,
  }

@jsonmemoized
def extract_tensecondlevel_dataset_for_training():
  return extract_tensecondlevel_dataset_from_users(training_users)

@jsonmemoized
def extract_tensecondlevel_dataset_for_test():
  return extract_tensecondlevel_dataset_from_users(test_users)


In [7]:
#a=extract_secondlevel_training_data_from_user(training_users[0])
#print 'extracting dataset for training'
#extract_secondlevel_dataset_for_training()
#extract_tensecondlevel_dataset_for_training()
#print 'extracting dataset for test'
#extract_secondlevel_dataset_for_test()
#extract_tensecondlevel_dataset_for_test()
#print 'extraction done'

In [8]:
#print zipkeys({'a':[3,4,5], 'b':[6,7,8]}, 'a', 'b')

In [9]:
'''
@memoized
def total_usage_of_domains_in_training():
  return sum_values_in_list_of_dict([get_domain_to_time_spent_for_user(user)for user in training_users])

@memoized
def top_n_domains_by_usage(n=10):
  domain_to_usage = total_usage_of_domains_in_training()
  return [x[0] for x in sorted(domain_to_usage.items(), key=itemgetter(1), reverse=True)[:n]]
'''


'\n@memoized\ndef total_usage_of_domains_in_training():\n  return sum_values_in_list_of_dict([get_domain_to_time_spent_for_user(user)for user in training_users])\n\n@memoized\ndef top_n_domains_by_usage(n=10):\n  domain_to_usage = total_usage_of_domains_in_training()\n  return [x[0] for x in sorted(domain_to_usage.items(), key=itemgetter(1), reverse=True)[:n]]\n'

In [10]:
#print top_n_domains_by_visits()

In [11]:
#print top_n_domains_by_usage(10)
#print top_n_domains_by_usage(5)

In [12]:
#print domain_to_id('newtab')

In [13]:
'''
def dataset_to_feature_vectors(dataset):
  topdomains = numpy.array([domain_to_id(x) for x in top_n_domains_by_visits(20)])
  num_features = 3 + 2*len(topdomains) + 2*len(get_rescuetime_productivity_levels())
  output = [[0]*num_features for x in xrange(len(dataset['sinceprev']))]
  #output = numpy.zeros((len(dataset['sinceprev']), num_features), dtype=object) # object instead of float, so we can have floats and ints
  for idx,sinceprev,tonext,fromdomain,todomain in zipkeys_idx(dataset, 'sinceprev', 'tonext', 'fromdomain', 'todomain'):
    cur = output[idx]
    cur[0] = sinceprev
    cur[1] = tonext
    cur[2] = int(fromdomain == todomain)
    feature_num = 3
    for domain_idx,domain in enumerate(topdomains):
      cur[feature_num+domain_idx] = int(fromdomain == domain)
    feature_num += len(topdomains)
    for domain_idx,domain in enumerate(topdomains):
      cur[feature_num+domain_idx] = int(todomain == domain)
    feature_num += len(topdomains)
    fromdomain_name = id_to_domain(fromdomain)
    todomain_name = id_to_domain(todomain)
    fromdomain_productivity = domain_to_productivity(fromdomain_name)
    todomain_productivity = domain_to_productivity(todomain_name)
    for productivity_idx,productivity in enumerate(get_rescuetime_productivity_levels()):
      cur[feature_num+productivity_idx] = int(fromdomain_productivity == productivity)
    feature_num += len(get_rescuetime_productivity_levels())
    for productivity_idx,productivity in enumerate(get_rescuetime_productivity_levels()):
      cur[feature_num+productivity_idx] = int(todomain_productivity == productivity)
    feature_num += len(get_rescuetime_productivity_levels())
  return output
'''

def remove_cached_features():
  os.remove('get_test_feature_vector.msgpack')
  os.remove('get_training_feature_vector.msgpack')
  #os.remove('get_test_labels.msgpack')
  #os.remove('get_training_labels.msgpack')


In [14]:
#remove_cached_features()

In [15]:
'''
@msgpackmemoized
def get_training_feature_vector():
  return dataset_to_feature_vectors(extract_tensecondlevel_dataset_for_training())

@msgpackmemoized
def get_training_labels():
  return extract_tensecondlevel_dataset_for_training()['label']

@msgpackmemoized
def get_test_feature_vector():
  return dataset_to_feature_vectors(extract_tensecondlevel_dataset_for_test())

@msgpackmemoized
def get_test_labels():
  return extract_tensecondlevel_dataset_for_test()['label']
'''

"\n@msgpackmemoized\ndef get_training_feature_vector():\n  return dataset_to_feature_vectors(extract_tensecondlevel_dataset_for_training())\n\n@msgpackmemoized\ndef get_training_labels():\n  return extract_tensecondlevel_dataset_for_training()['label']\n\n@msgpackmemoized\ndef get_test_feature_vector():\n  return dataset_to_feature_vectors(extract_tensecondlevel_dataset_for_test())\n\n@msgpackmemoized\ndef get_test_labels():\n  return extract_tensecondlevel_dataset_for_test()['label']\n"

In [4]:
'''
def train_classifier_on_data(training_data):
  #classifier = sklearn.tree.DecisionTreeClassifier(max_depth=2) # .71 on test
  #classifier = sklearn.tree.DecisionTreeClassifier(max_depth=1)
  #classifier = sklearn.tree.DecisionTreeClassifier()
  #classifier = sklearn.naive_bayes.GaussianNB() # .73 on test
  #classifier = sklearn.svm.LinearSVC()
  #classifier = sklearn.linear_model.SGDClassifier(class_weight='balanced')
  classifier = sklearn.linear_model.SGDClassifier(loss='modified_huber') # .73 on test
  classifier.fit(dataset_to_feature_vectors(training_data), numpy.array(training_data['label']))
  return classifier

def get_classifier():
  return train_classifier_on_data(extract_tensecondlevel_dataset_for_training())
'''

global_feature_filter = '1'*53
#global_feature_filter = '11100000000000000000000000000000000000000000000000000'

def get_feature_filter():
  return global_feature_filter
  return ''.join(map(str, selected_features))

is_second = len(sys.argv) > 1 and sys.argv[1] == 'second'

@memoized
def get_filtered_features_train():
  selected_features = get_feature_filter()
  if is_second:
    return numpy.array(get_feature_vector_for_secondlevel_train(selected_features))
  return numpy.array(get_feature_vector_for_tensecondlevel_train(selected_features))

@memoized
def get_filtered_features_test():
  selected_features = get_feature_filter()
  if is_second:
    return numpy.array(get_feature_vector_for_secondlevel_test(selected_features))
  return numpy.array(get_feature_vector_for_tensecondlevel_test(selected_features))

@memoized
def get_test_labels():
  if is_second:
    return numpy.array(get_labels_for_secondlevel_test())
  return numpy.array(get_labels_for_tensecondlevel_test())

@memoized
def get_training_labels():
  if is_second:
    return numpy.array(get_labels_for_secondlevel_train())
  return numpy.array(get_labels_for_tensecondlevel_train())

#def get_labels_for_user(user):
#  return get_tensecondlevel_activespan_labels_for_user(user)

'''
# we normally want to use the tensecond level ones
def get_filtered_features_train():
  selected_features = get_feature_filter()
  return get_feature_vector_for_secondlevel_train(selected_features)

def get_filtered_features_test():
  selected_features = get_feature_filter()
  return get_feature_vector_for_secondlevel_test(selected_features)

def get_test_labels():
  return get_labels_for_secondlevel_test()

def get_training_labels():
  return get_labels_for_secondlevel_train()

def get_labels_for_user(user):
  return get_secondlevel_activespan_labels_for_user(user)
'''


'''
def filter_features(arr):
  # from get_selected_features()
  selected_features = get_feature_filter()
  #selected_feature_idx = [i for i,x in enumerate(selected_features) if x]
  #return arr[:,selected_feature_idx]
  output = []
  for line in arr:
    output.append([line[i] for i,x in enumerate(selected_features) if x])
    #output.append([x for i,x in enumerate(line) if selected_features[i]])
  return output
'''

classifier_algorithm = xgboost.XGBClassifier
xgboost_params = {}

def get_classifier():
  #classifier = sklearn.naive_bayes.GaussianNB()
  #classifier = sklearn.linear_model.SGDClassifier(loss='modified_huber') # .73 on test
  #classifier = sklearn.linear_model.SGDClassifier()
  #classifier = sklearn.ensemble.RandomForestClassifier()
  classifier = xgboost.XGBClassifier()
  classifier.set_params(**xgboost_params)
  #classifier.train()
  #classifier = sklearn.ensemble.AdaBoostClassifier()
  #classifier = sklearn.ensemble.GradientBoostingClassifier()
  #classifier = classifier_algorithm()
  classifier.fit(get_filtered_features_train(), get_training_labels())
  return classifier

NameError: name 'sklearn' is not defined

In [18]:
#a=get_filtered_features_test()

In [19]:
#print len(a)

In [20]:
#b=get_test_labels()

In [21]:
#print len(b)

In [22]:
#print len(get_feature_filter())

In [23]:
#print len(get_training_feature_vector()[0])

In [24]:
'''
def make_predictions_with_classifier_on_dataset(classifier, dataset):
  return classifier.predict(dataset_to_feature_vectors(dataset))

def make_proba_predictions_with_classifier_on_dataset(classifier, dataset):
  return [x[1] for x in classifier.predict_proba(dataset_to_feature_vectors(dataset))]

def evaluate_classifier(classifier):
  test_predictions = make_predictions_with_classifier_on_dataset(classifier, test_data)
  print sklearn.metrics.classification_report(test_data['label'], test_predictions)
'''

def make_predictions_with_classifier_on_test(classifier):
  #return classifier.predict(filter_features(numpy.array(get_test_feature_vector())))
  return classifier.predict(get_filtered_features_test())

def make_predictions_with_classifier_on_train(classifier):
  #return classifier.predict(filter_features(numpy.array(get_test_feature_vector())))
  return classifier.predict(get_filtered_features_train())


def evaluate_classifier(classifier):
  test_predictions = make_predictions_with_classifier_on_test(classifier)
  print sklearn.metrics.classification_report(get_test_labels(), test_predictions)

def evaluate_classifier_train(classifier):
  train_predictions = make_predictions_with_classifier_on_train(classifier)
  print sklearn.metrics.classification_report(get_training_labels(), test_predictions)
  
def evaluate_classifier_for_user(classifier, user):
  dataset = extract_secondlevel_dataset_from_user(user, True)
  test_labels = get_labels_for_user(user)
  

In [25]:
#a= get_training_feature_vector()

"\npickle_file = 'classifier_threefeatures_randomforest.pickle'\nif path.exists(pickle_file):\n  classifier = pickle.load(open(pickle_file))\nelse:\n  classifier = get_classifier()\n  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)\n\nevaluate_classifier(classifier)\n"

In [28]:
'''
global_feature_filter = '1'*53
#classifier_algorithm = xgboost.XGBClassifier()
xgboost_params = {}

pickle_file = 'classifier_allfeatures_xgboost.pickle'
if path.exists(pickle_file):
  classifier = pickle.load(open(pickle_file))
else:
  classifier = get_classifier()
  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)

evaluate_classifier(classifier)
'''

             precision    recall  f1-score   support

      False       0.83      0.81      0.82   1295568
       True       0.83      0.84      0.83   1383871

avg / total       0.83      0.83      0.83   2679439



In [ ]:
'''
xgboost_params = {'objective': "binary:logistic"}

pickle_file = 'classifier_allfeatures_xgboost_binary_logistic.pickle'
if path.exists(pickle_file):
  classifier = pickle.load(open(pickle_file))
else:
  classifier = get_classifier()
  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)

evaluate_classifier(classifier)
'''


In [ ]:
xgboost_params = {'objective': "binary:logistic"}
for colsample_bytree in [1.0, 0.9, 0.8, 0.7, 0.6, 0.5]:
  for subsample in shuffled([0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 1.0]):
    xgboost_params['subsample'] = subsample
    xgboost_params['colsample_bytree'] = colsample_bytree
    features_string = '_'.join(map(str, ['subsample', subsample, 'colsample_bytree', colsample_bytree]))
    if is_second:
      classifier_filename = 'xgboost_second_allfeatures_' + features_string + '.pickle'
    else:
      classifier_filename = 'xgboost_tensecond_allfeatures_' + features_string + '.pickle'
    if path.exists(classifier_filename):
      continue
    print classifier_filename
    try:
      classifier = get_classifier()
      pickle.dump(classifier, open(classifier_filename, 'w'), pickle.HIGHEST_PROTOCOL)
      evaluate_classifier(classifier)
    except:
      traceback.print_exc()
      continue

In [29]:
'''
xgboost_params = {'max_depth': 4, 'num_parallel_tree': 1000, 'subsample': 0.5, 'colsample_bytree': 0.5, 'nround': 1, 'objective': "binary:logistic"}

pickle_file = 'classifier_allfeatures_xgboost_randomforest.pickle'
if path.exists(pickle_file):
  classifier = pickle.load(open(pickle_file))
else:
  classifier = get_classifier()
  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)

evaluate_classifier(classifier)
'''

feature_vector_for_tensecondlevel_insession_train/11111111111111111111111111111111111111111111111111111.json
feature_vector_for_tensecondlevel_insession_test/11111111111111111111111111111111111111111111111111111.json
             precision    recall  f1-score   support

      False       0.84      0.85      0.85   1295568
       True       0.86      0.85      0.85   1383871

avg / total       0.85      0.85      0.85   2679439



"\npickle_file = 'classifier_threefeatures_randomforest_v2.pickle'\nif path.exists(pickle_file):\n  classifier = pickle.load(open(pickle_file))\nelse:\n  classifier = get_classifier()\n  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)\n\nevaluate_classifier(classifier)\n"

"\npickle_file = 'classifier_allfeatures_randomforest.pickle'\nif path.exists(pickle_file):\n  classifier = pickle.load(open(pickle_file))\nelse:\n  classifier = get_classifier()\n  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)\n\nevaluate_classifier(classifier)\n"

"\npickle_file = 'classifier_allfeatures_randomforest_v2.pickle'\nif path.exists(pickle_file):\n  classifier = pickle.load(open(pickle_file))\nelse:\n  classifier = get_classifier()\n  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)\n\nevaluate_classifier(classifier)\n"

"\nprint 'get_selected_features_chi2'\nprint datetime.datetime.fromtimestamp(time.time())\nprint get_selected_features_chi2()\nprint 'get_selected_features_rfe'\nprint datetime.datetime.fromtimestamp(time.time())\nprint get_selected_features_rfe()\nprint 'get_selected_features_rfecv'\nprint datetime.datetime.fromtimestamp(time.time())\nprint get_selected_features_rfecv()\n#print get_selected_features_rfe()\n"

'\nprecision_all,recall_all,_ = sklearn.metrics.precision_recall_curve(get_test_labels(), test_predictions)\n\nbest_f1 = 0.0\nprecision = 0.0\nrecall = 0.0\nfor x,y in zip(precision_all,recall_all):\n  f1 = 2*(x*y)/(x+y)\n  if f1 > best_f1:\n    best_f1 = f1\n    precision = x\n    recall = y\nprint best_f1, precision, recall\n'